In [56]:
print('Started...')
import glob
import pandas as pd
import pydicom
import os
import matplotlib.pyplot as plt
import png
import time
import numpy as np
import os
from fnmatch import fnmatch


# Gobals
DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/dcm/malignant/1.2.840.113681.2230565232.95*'
#DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/dcm/normal/1.2.840.113681.2230565232.95*'
DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/IMAGE_DATABASE/PUBLIC_SHARE/IMAGES/STANDARD_SET/1/optm10'
DICOM_FILES = '/vol/vssp/cvpwrkspc01/scratch/wm0015/download/IMAGE_DATABASE/PUBLIC_SHARE/IMAGES/STANDARD_SET/1'


# File walk method
def getFiles():
    fMal = []
    for path, subdirs, files in os.walk(DICOM_FILES):
        for name in files:
            if fnmatch(name, '*.dcm'):
                fMal.append(os.path.join(path, name))

    print(len(fMal), ' dicom images found')


    # Load dicom files into np array
    malignant = np.array([])
    dicomImg = {'malignant':np.array([]), 'normal':np.array([])}
    count = 0
    print(len(fMal), ' Malignant found')
    print('Loading images...')
    for f in fMal:
        dicomImg['malignant'] = np.append(dicomImg['malignant'], pydicom.dcmread(f))
        count += 1
        print(count, '/', len(fMal))
    return dicomImg, fMal



print('name:\n', dicomImg['malignant'][0].PresentationIntentType)

def deletePreProcessed(dicomImg, fMal):
    forPresentationCount = 0
    toDelete = []
    for index, _ in enumerate(dicomImg['malignant']):
        tmp = _.PresentationIntentType
        if tmp == 'FOR PRESENTATION':
            forPresentationCount += 1
        else:
            toDelete.append(index)
    print('dicomImg len:', len(dicomImg['malignant']))
    np.delete(dicomImg['malignant'], toDelete)
    mask = np.ones(len(dicomImg['malignant']), dtype=bool)
    mask[toDelete] = False
    dicomImg['malignant'] = dicomImg['malignant'][mask]
    fMal = np.asarray(fMal)
    fMal = fMal[mask]
    print('\nTotal DICOM: ', len(dicomImg['malignant']))
    print('Total FOR PROCESSING: ', forPresentationCount)
    return dicomImg, fMal

   
# Import xls file, extract ROI coords, get pixel array from DICOM image
def buildDict(dicomImg, fMal):
    xls = pd.ExcelFile('/vol/vssp/cvpwrkspc01/scratch/wm0015/download/batch_1_IMAGE.xls')
    sheet = xls.parse(1)

    # Create a dict where the key is the image name
    # Each key has the image, and coords
    img = {}
    for i in range(len(fMal)):
        key = os.path.basename(fMal[i])[:-4]
        # Find row in the xml file that holds the img info
        try:
            indx = [_==key for _ in sheet['ImageSOPIUID']].index(True) # ImageSOPIUID, ReferencedSOPInstanceUID
            img.update({key:{}})
            img[key].update({'img': dicomImg['malignant'][i].pixel_array})
            img[key].update({'x': [sheet['X1'][indx], sheet['X2'][indx]]})
            img[key].update({'y': [sheet['Y1'][indx], sheet['Y2'][indx]]})
        except ValueError:
            print('MY_ERROR: key not found:\n', key)

    print(len(img), 'DICOM images extracted')
    # print(img)
    # Crop the images to given ROI
    toDelete = [] # Keep track of error causing keys and delete after loop
    for key in img:
        try:
            tmp = img[key]['img']
            x = img[key]['x']
            y = img[key]['y']
            x = [int(x[0]), int(x[1])]
            y = [int(y[0]), int(y[1])]
            #x is width, y is height
            #in numpy array, y,x
            img[key].update({'cropROI':tmp[y[0]:y[1], x[0]:x[1]]}) 
        except:
            print('ROI extraction failed...Removing key\nkey:  ',key, '\nx  :', x,'\ny:  ',y)
            toDelete.append(key)

    #img_calc = img['1.2.840.113681.2230565232.954.3504500766.32'] 
    for _ in toDelete:
        img.pop(_)
    return img


# Write images to disk with markers and basic crop    
def writeMarkedImages(img): 
    for key in img:
        plt.figure(figsize=(20,20))
        marker = [(img[key]['x'][1] + img[key]['x'][0])/2, (img[key]['y'][1] + img[key]['y'][0])/2 ]   
        plt.imshow(img[key]['img']/16383, cmap='gray', vmin=0, vmax=0.2)
        plt.plot(marker[0], marker[1], marker='x', color=[1,0,1], markersize=30)
        plt.savefig('/vol/vssp/cvpwrkspc01/scratch/wm0015/markers/tmp/' + key +'_full.png')
        #plt.show()
        plt.close()

        plt.figure(figsize=(20,20))
        plt.imshow(img[key]['cropROI'], cmap='gray') 
        plt.savefig('/vol/vssp/cvpwrkspc01/scratch/wm0015/markers/tmp/' + key +'_crop.png')   
        #plt.show()
        plt.close()

# Crop the images so that the ROI is centred but all crops are the same size
def computeCrops(img):
    cS = 256
    for key in img:
        tmp = img[key]['img']
        x = img[key]['x']
        y = img[key]['y']
        c = [round((x[0]+x[1])/2), round((y[0]+y[1])/2)]
        # Pad images before cropping (wrap around)
        pad = 1000
        tmp = np.pad(tmp, pad, mode='wrap')
        img[key].update({'crop': tmp[int(c[1]-cS/2+pad):int(c[1]+cS/2+pad), int(c[0]-cS/2+pad):int(c[0]+cS/2+pad)]})
        # Reshape from (256, 256) to (256, 256, 1)
        img[key]['crop'] = np.reshape(img[key]['crop'],(img[key]['crop'].shape[0], img[key]['crop'].shape[1], 1))
    return img
    
# Find bit depth
def findBitDepth(img):
    maxmax = 0
    for key in img:
        tmp = img[key]['img']
        print(np.amax(tmp))
        if np.amax(tmp) > maxmax:
            maxmax = np.amax(tmp)
    print('The largest value is: ', maxmax)
    
# Find average ROI size
def findAverageROISize(img):
    totalX = 0
    totalY = 0
    for key in img:
        x = img[key]['x']
        y = img[key]['y']
        totalX += x[1] - x[0]
        totalY += y[1] - y[0]
    print('Average ROI width: ', totalX/len(img), '\nAverage ROI length: ', totalY/len(img))
    
# View crops / save to disk
def writeCropsToDisk(img):
    count = 0
    for key in img:
        count+=1
        f = open('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch1_crop/' + key + '.png', 'wb')
        w = png.Writer(width = cS, height = cS, bitdepth=16, greyscale=True)
        w.write(f, img[key]['crop'])
        f.close()
        print(count, '/', len(img))
        if count == -1:
            break
            
def buildArrayForPickle(img):
    allCrops = []
    for key in img:
        allCrops.append(img[key]['crop'])
    return allCrops
        
def savePickle(ob):
    print('Pickling...')
    with open('/vol/vssp/cvpwrkspc01/scratch/wm0015/batch1_crop256.pickle', 'wb') as output:
        pickle.dump(ob, output, pickle.HIGHEST_PROTOCOL)
        
def getNormals


def main():
    dicomImg, fileList = getFiles()
    dicomImg, fileList = deletePreProcessed(dicomImg, fileList)
    img = buildDict(dicomImg, fileList)
    img = computeCrops(img)
    writeMarkedImages(img)
    findBitDepth(img)
    writeCropsToDisk(img)
    savePickle(buildArrayForPickle(img)) 
    findAverageROISize(img)


if __name__ == "__main__":
    main()

print('Done')

Started...
name:
 FOR PRESENTATION
2692  dicom images found
2692  Malignant found
Loading images...
1 / 2692
2 / 2692
3 / 2692
4 / 2692
5 / 2692
6 / 2692
7 / 2692
8 / 2692
9 / 2692
10 / 2692
11 / 2692
12 / 2692
13 / 2692
14 / 2692
15 / 2692
16 / 2692
17 / 2692
18 / 2692
19 / 2692
20 / 2692
21 / 2692
22 / 2692
23 / 2692
24 / 2692
25 / 2692
26 / 2692
27 / 2692
28 / 2692
29 / 2692
30 / 2692
31 / 2692
32 / 2692
33 / 2692
34 / 2692
35 / 2692
36 / 2692
37 / 2692
38 / 2692
39 / 2692
40 / 2692
41 / 2692
42 / 2692
43 / 2692
44 / 2692
45 / 2692
46 / 2692
47 / 2692
48 / 2692
49 / 2692
50 / 2692
51 / 2692
52 / 2692
53 / 2692
54 / 2692
55 / 2692
56 / 2692
57 / 2692
58 / 2692
59 / 2692
60 / 2692
61 / 2692
62 / 2692
63 / 2692
64 / 2692
65 / 2692
66 / 2692
67 / 2692
68 / 2692
69 / 2692
70 / 2692
71 / 2692
72 / 2692
73 / 2692
74 / 2692
75 / 2692
76 / 2692
77 / 2692
78 / 2692
79 / 2692
80 / 2692
81 / 2692
82 / 2692
83 / 2692
84 / 2692
85 / 2692
86 / 2692
87 / 2692
88 / 2692
89 / 2692
90 / 2692
91 / 2692

747 / 2692
748 / 2692
749 / 2692
750 / 2692
751 / 2692
752 / 2692
753 / 2692
754 / 2692
755 / 2692
756 / 2692
757 / 2692
758 / 2692
759 / 2692
760 / 2692
761 / 2692
762 / 2692
763 / 2692
764 / 2692
765 / 2692
766 / 2692
767 / 2692
768 / 2692
769 / 2692
770 / 2692
771 / 2692
772 / 2692
773 / 2692
774 / 2692
775 / 2692
776 / 2692
777 / 2692
778 / 2692
779 / 2692
780 / 2692
781 / 2692
782 / 2692
783 / 2692
784 / 2692
785 / 2692
786 / 2692
787 / 2692
788 / 2692
789 / 2692
790 / 2692
791 / 2692
792 / 2692
793 / 2692
794 / 2692
795 / 2692
796 / 2692
797 / 2692
798 / 2692
799 / 2692
800 / 2692
801 / 2692
802 / 2692
803 / 2692
804 / 2692
805 / 2692
806 / 2692
807 / 2692
808 / 2692
809 / 2692
810 / 2692
811 / 2692
812 / 2692
813 / 2692
814 / 2692
815 / 2692
816 / 2692
817 / 2692
818 / 2692
819 / 2692
820 / 2692
821 / 2692
822 / 2692
823 / 2692
824 / 2692
825 / 2692
826 / 2692
827 / 2692
828 / 2692
829 / 2692
830 / 2692
831 / 2692
832 / 2692
833 / 2692
834 / 2692
835 / 2692
836 / 2692
837 / 2692

1453 / 2692
1454 / 2692
1455 / 2692
1456 / 2692
1457 / 2692
1458 / 2692
1459 / 2692
1460 / 2692
1461 / 2692
1462 / 2692
1463 / 2692
1464 / 2692
1465 / 2692
1466 / 2692
1467 / 2692
1468 / 2692
1469 / 2692
1470 / 2692
1471 / 2692
1472 / 2692
1473 / 2692
1474 / 2692
1475 / 2692
1476 / 2692
1477 / 2692
1478 / 2692
1479 / 2692
1480 / 2692
1481 / 2692
1482 / 2692
1483 / 2692
1484 / 2692
1485 / 2692
1486 / 2692
1487 / 2692
1488 / 2692
1489 / 2692
1490 / 2692
1491 / 2692
1492 / 2692
1493 / 2692
1494 / 2692
1495 / 2692
1496 / 2692
1497 / 2692
1498 / 2692
1499 / 2692
1500 / 2692
1501 / 2692
1502 / 2692
1503 / 2692
1504 / 2692
1505 / 2692
1506 / 2692
1507 / 2692
1508 / 2692
1509 / 2692
1510 / 2692
1511 / 2692
1512 / 2692
1513 / 2692
1514 / 2692
1515 / 2692
1516 / 2692
1517 / 2692
1518 / 2692
1519 / 2692
1520 / 2692
1521 / 2692
1522 / 2692
1523 / 2692
1524 / 2692
1525 / 2692
1526 / 2692
1527 / 2692
1528 / 2692
1529 / 2692
1530 / 2692
1531 / 2692
1532 / 2692
1533 / 2692
1534 / 2692
1535 / 2692
1536

2136 / 2692
2137 / 2692
2138 / 2692
2139 / 2692
2140 / 2692
2141 / 2692
2142 / 2692
2143 / 2692
2144 / 2692
2145 / 2692
2146 / 2692
2147 / 2692
2148 / 2692
2149 / 2692
2150 / 2692
2151 / 2692
2152 / 2692
2153 / 2692
2154 / 2692
2155 / 2692
2156 / 2692
2157 / 2692
2158 / 2692
2159 / 2692
2160 / 2692
2161 / 2692
2162 / 2692
2163 / 2692
2164 / 2692
2165 / 2692
2166 / 2692
2167 / 2692
2168 / 2692
2169 / 2692
2170 / 2692
2171 / 2692
2172 / 2692
2173 / 2692
2174 / 2692
2175 / 2692
2176 / 2692
2177 / 2692
2178 / 2692
2179 / 2692
2180 / 2692
2181 / 2692
2182 / 2692
2183 / 2692
2184 / 2692
2185 / 2692
2186 / 2692
2187 / 2692
2188 / 2692
2189 / 2692
2190 / 2692
2191 / 2692
2192 / 2692
2193 / 2692
2194 / 2692
2195 / 2692
2196 / 2692
2197 / 2692
2198 / 2692
2199 / 2692
2200 / 2692
2201 / 2692
2202 / 2692
2203 / 2692
2204 / 2692
2205 / 2692
2206 / 2692
2207 / 2692
2208 / 2692
2209 / 2692
2210 / 2692
2211 / 2692
2212 / 2692
2213 / 2692
2214 / 2692
2215 / 2692
2216 / 2692
2217 / 2692
2218 / 2692
2219

 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.6266.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.6274.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.6496.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.6484.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.682981803.9661.1516806470059.86.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.682981803.9661.1516806470059.98.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.68298180.9661.1516806470059.132.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.68298180.9661.1516806470059.112.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.68298180.9661.1516806470059.115.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.68298180.9661.1516806470059.142.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.68298180.9661.1516806470059.137.0
MY_ERROR: key not found:

 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.3913.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.3917.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.3905.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.3909.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.3440.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.3436.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.3409.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.3421.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.6133.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.6129.0
MY_ERROR: key not found:
 1.2.826.0.1.3680043.9.3218.1.1.6829818.9661.1516806470059.6123.0
MY_ERROR: key not found:

4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4016
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
3875
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
3287
3434
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095
4095


In [15]:
print('hello world')
import pickle

hello world


In [28]:

def test():
    print(x)
    x = x+1
    
x = 2
test()


UnboundLocalError: local variable 'x' referenced before assignment